In [ ]:
# sns.pairplot(df)
# plt.show()

## Basic Visualization

In [27]:
# Check missing values
missing_values = df.isnull().sum()
print("Missing Values by Column:")
print(missing_values)
print("\n" + "="*50)
print("Missing Values (Non-Zero Only):")
print(missing_values[missing_values > 0])
print("\n" + "="*50)
print(f"Total Missing Percentage: {(df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100):.2f}%")

Missing Values by Column:
Country             0
Indicator           0
Source              0
Unit               22
Currency        10503
Frequency           0
Country Code        0
Time                4
Amount             59
dtype: int64

Missing Values (Non-Zero Only):
Unit           22
Currency    10503
Time            4
Amount         59
dtype: int64

Total Missing Percentage: 4.95%


In [30]:
# Map indicators to SDGs
sdg_mapping = {
    'SDG 1 (No Poverty)': ['GDP per Capita', 'GDP per capita', 'GDP per Capita ', 'Real GDP', 'Nominal GDP'],
    'SDG 2 (Zero Hunger)': ['Food Inflation', 'Food Inflation ', 'Food Inflation YoY'],
    'SDG 3 (Good Health)': ['Health Expenditure', 'Population'],
    'SDG 4 (Quality Education)': ['Education Expenditure'],
    'SDG 8 (Decent Work & Economic Growth)': ['GDP Growth Rate', 'Unemployment Rate', 'Tax Revenue', 
                                               'Revenue', 'Exports', 'Imports', 'Labour Force'],
    'SDG 9 (Industry & Infrastructure)': ['Capital Expenditure', 'Government Debt'],
    'SDG 10 (Reduced Inequality)': ['Budget Deficit/Surplus', 'Expenditure'],
    'SDG 16 (Peace & Strong Institutions)': ['Defence Expenditure', 'Value Added Tax (VAT)', 
                                              'Interest Rate', 'Inflation Rate', 'Inflation Rate ',
                                              'Consumer Price Index (CPI)']
}

# Calculate data completeness per SDG
print("="*70)
print("SDG DATA COMPLETENESS ANALYSIS")
print("="*70)

sdg_scores = {}
for sdg, indicators in sdg_mapping.items():
    # Get rows for this SDG's indicators
    sdg_data = df[df['Indicator'].isin(indicators)]
    
    if len(sdg_data) > 0:
        # Calculate completeness (non-null Amount values)
        completeness = (sdg_data['Amount'].notna().sum() / len(sdg_data)) * 100
        sdg_scores[sdg] = {
            'completeness': completeness,
            'total_rows': len(sdg_data),
            'valid_values': sdg_data['Amount'].notna().sum(),
            'indicators': indicators
        }
        
        print(f"\n{sdg}")
        print(f"  Completeness: {completeness:.2f}%")
        print(f"  Total Rows: {len(sdg_data):,}")
        print(f"  Valid Values: {sdg_data['Amount'].notna().sum():,}")
        print(f"  Indicators: {len(indicators)}")
    else:
        print(f"\n{sdg}: ❌ No data found")

print("\n" + "="*70)
print("RANKING BY DATA COMPLETENESS:")
print("="*70)
sorted_sdgs = sorted(sdg_scores.items(), key=lambda x: x[1]['completeness'], reverse=True)
for rank, (sdg, data) in enumerate(sorted_sdgs, 1):
    print(f"{rank}. {sdg}: {data['completeness']:.2f}%")

SDG DATA COMPLETENESS ANALYSIS

SDG 1 (No Poverty)
  Completeness: 100.00%
  Total Rows: 2,090
  Valid Values: 2,090
  Indicators: 5

SDG 2 (Zero Hunger)
  Completeness: 98.82%
  Total Rows: 1,099
  Valid Values: 1,086
  Indicators: 3

SDG 3 (Good Health)
  Completeness: 100.00%
  Total Rows: 366
  Valid Values: 366
  Indicators: 2

SDG 4 (Quality Education)
  Completeness: 100.00%
  Total Rows: 19
  Valid Values: 19
  Indicators: 1

SDG 8 (Decent Work & Economic Growth)
  Completeness: 99.79%
  Total Rows: 7,027
  Valid Values: 7,012
  Indicators: 7

SDG 9 (Industry & Infrastructure)
  Completeness: 100.00%
  Total Rows: 1,784
  Valid Values: 1,784
  Indicators: 2

SDG 10 (Reduced Inequality)
  Completeness: 99.58%
  Total Rows: 2,378
  Valid Values: 2,368
  Indicators: 2

SDG 16 (Peace & Strong Institutions)
  Completeness: 99.77%
  Total Rows: 9,021
  Valid Values: 9,000
  Indicators: 6

RANKING BY DATA COMPLETENESS:
1. SDG 1 (No Poverty): 100.00%
2. SDG 3 (Good Health): 100.00%
3. 

## 🎯 SDG Mapping & Data Completeness Analysis

In [29]:
# Check if Amount is numeric or text
print("Amount column data type:", df['Amount'].dtype)
print("\nSample Amount values:")
print(df['Amount'].head(20))

# Try to identify non-numeric values
if df['Amount'].dtype == 'object':
    print("\n⚠️ Amount column is TEXT. Checking for non-numeric values...")
    non_numeric = df[pd.to_numeric(df['Amount'], errors='coerce').isna() & df['Amount'].notna()]
    print(f"Found {len(non_numeric)} non-numeric values in Amount column.")
    if len(non_numeric) > 0:
        print("\nSample non-numeric values:")
        print(non_numeric['Amount'].head(10).tolist())

Amount column data type: object

Sample Amount values:
0      -56213
1      -67563
2      -72379
3      -98886
4     -134138
5     -165689
6     -246853
7     -254715
8     -284314
9         NaN
10    -366401
11    -425385
12    -423019
13    -408509
14    -459449
15    -477486
16    -580182
17       -962
18     -11097
19     -29076
Name: Amount, dtype: object

⚠️ Amount column is TEXT. Checking for non-numeric values...
Found 443 non-numeric values in Amount column.

Sample non-numeric values:
['\xa0103.60', '\xa0105.45', '107.65\xa0', '109.84\xa0', '114.45\xa0', '\xa0127.03', '3,883', '3,449', '3,833', '2,900']


In [28]:
# Check for duplicate indicators (with trailing spaces)
print("Checking for duplicate indicators with spacing issues:")
indicators = df['Indicator'].unique()
indicators_stripped = [ind.strip() for ind in indicators]

if len(indicators) != len(set(indicators_stripped)):
    print("⚠️ WARNING: Found duplicate indicators with spacing issues!")
    duplicates = {}
    for ind in indicators:
        stripped = ind.strip()
        if stripped not in duplicates:
            duplicates[stripped] = []
        duplicates[stripped].append(ind)
    
    print("\nDuplicates found:")
    for key, values in duplicates.items():
        if len(values) > 1:
            print(f"  '{key}': {values}")
else:
    print("✅ No duplicate indicators found.")

print("\n" + "="*50)
print(f"Total unique indicators: {len(indicators)}")
print(f"Total unique countries: {df['Country'].nunique()}")

Checking for duplicate indicators with spacing issues:
⚠️ WARNING: Found duplicate indicators with spacing issues!

Duplicates found:
  'GDP per Capita': ['GDP per Capita', 'GDP per Capita ']
  'Inflation Rate': ['Inflation Rate', 'Inflation Rate ']

Total unique indicators: 27
Total unique countries: 14


## Data Quality Checks

## Missing Values Analysis

In [26]:
# Statistical summary
df.describe(include='all')

,Country,Indicator,Source,Unit,Currency,Frequency,Country Code,Time,Amount
count,23784,23784,23784,23762,13281,23784,23784,23780,23725
unique,14,27,28,10,13,3,14,1070,15126
top,South Africa,Inflation Rate,Central Bank of West African States,Million,ZAR,Monthly,ZAF,2021-01-01,12
freq,5108,2879,3295,9964,3086,16850,5108,246,222


In [25]:
# Data types and basic info
print("Data Types and Non-Null Counts:")
df.info()
print("\n" + "="*50)
print("Unique Indicators:")
print(df['Indicator'].unique())

Data Types and Non-Null Counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23784 entries, 0 to 23783
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country       23784 non-null  object
 1   Indicator     23784 non-null  object
 2   Source        23784 non-null  object
 3   Unit          23762 non-null  object
 4   Currency      13281 non-null  object
 5   Frequency     23784 non-null  object
 6   Country Code  23784 non-null  object
 7   Time          23780 non-null  object
 8   Amount        23725 non-null  object
dtypes: object(9)
memory usage: 1.6+ MB

Unique Indicators:
['Budget Deficit/Surplus' 'Capital Expenditure'
 'Consumer Price Index (CPI)' 'Defence Expenditure'
 'Education Expenditure' 'Expenditure' 'Exports' 'Food Inflation '
 'Food Inflation YoY' 'GDP Growth Rate' 'GDP per Capita' 'GDP per capita'
 'GDP per Capita ' 'Government Debt' 'Health Expenditure' 'Imports'
 'Inflation Rate' 'Inflation 

## Data Inspection
Check the structure, data types, and missing values.

In [24]:
# Load the fiscal data
df = pd.read_csv('../data/raw/10Alytics Hackathon- Fiscal Data.csv')

print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nFirst 5 rows:")
df.head()

Dataset Shape: (23784, 9)

Column Names:
['Country', 'Indicator', 'Source', 'Unit', 'Currency', 'Frequency', 'Country Code', 'Time', 'Amount']

First 5 rows:


,Country,Indicator,Source,Unit,Currency,Frequency,Country Code,Time,Amount
0,Egypt,Budget Deficit/Surplus,Central Bank of Egypt,Million,EGP,Yearly,EGY,2007-01-01,-56213
1,Egypt,Budget Deficit/Surplus,Central Bank of Egypt,Million,EGP,Yearly,EGY,2008-01-01,-67563
2,Egypt,Budget Deficit/Surplus,Central Bank of Egypt,Million,EGP,Yearly,EGY,2009-01-01,-72379
3,Egypt,Budget Deficit/Surplus,Central Bank of Egypt,Million,EGP,Yearly,EGY,2010-01-01,-98886
4,Egypt,Budget Deficit/Surplus,Central Bank of Egypt,Million,EGP,Yearly,EGY,2011-01-01,-134138


## Load Data
Load the raw data from the `data/raw` directory.

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set_theme(style="whitegrid")
%matplotlib inline

# Data Exploration
This notebook is for initial data exploration and understanding the dataset structure.